Deep Learning Assignment 2 Pujan

code reference: https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html

In [38]:

import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm


In [39]:

def get_device():
    return torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

get_device()


device(type='cuda', index=0)

In [62]:

class PujanNet(nn.Module):
    def __init__(self, data_set_name, batch_size, download_path, train_data_size, validation_data_size):
        super().__init__()
        torch.manual_seed(5)
    
        transform = transforms.Compose(
            [transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

        if data_set_name == "cifar_10":
            dataset = torchvision.datasets.CIFAR10(root=download_path, train=True, download=True, transform=transform)
            print('Length of Dataset ', len(dataset))

            train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_data_size, validation_data_size])
            print('Length of Train set ', len(train_dataset))
            print('Length of Val set ', len(val_dataset))
            self.trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
            self.valloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

            testset = torchvision.datasets.CIFAR10(root=download_path, train=False, download=True, transform=transform)
            print('Length of Testset ', len(testset))
            self.testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

            self.classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def display_input_images(self, number_of_images):
        # get some random training images
        dataiter = iter(self.trainloader)
        images, labels = dataiter.next()

        # show images
        imshow(torchvision.utils.make_grid(images))
        # print labels
        print(' '.join(f'{self.classes[labels[j]]:5s}' for j in range(number_of_images)))

    def display_ground_truth_images(self, number_of_images):
        dataiter = iter(self.testloader)
        images, labels = dataiter.next()

        # print images
        imshow(torchvision.utils.make_grid(images))
        print('GroundTruth: ', ' '.join(f'{self.classes[labels[j]]:5s}' for j in range(4)))


    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def train(self, epoch_number, learning_rate, momentum, criterion, optimizer_name):
        optimizer = optim.SGD(self.parameters(), lr= learning_rate, momentum=momentum)
        # if optimizer_name == "sgd": 

        train_results = []

        for epoch in range(epoch_number):  # loop over the dataset multiple times
            # Training phase
            running_loss = 0.0
            train_losses = []
            train_accuracies = []
            for i, data in enumerate(self.trainloader, 0):
                # get the inputs; data is a list of [inputs, labels]
                
                inputs, labels = data[0].to(get_device()), data[1].to(get_device())

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward + backward + optimize
                outputs = self(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                loss_item = loss.item()
                train_losses.append(loss_item)
                accuracy_score_batch = self.get_accuracy_score(outputs, labels)
                train_accuracies.append(accuracy_score_batch)
                # print statistics
                running_loss += loss_item
                if i % 2000 == 1999:    # print every 2000 mini-batches
                    print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
                    running_loss = 0.0

            
            train_loss_epoch = np.mean(train_losses)
            train_accuracy_epoch = 

            train_results.append({'epoch': epoch, 'train_loss': train_loss})

        print('Finished Training')
        return train_results

    def predictions(self, input_images):
        outputs = self(input_images)
        _, predicted = torch.max(outputs, 1)

        print('Predicted: ', ' '.join(f'{self.classes[predicted[j]]:5s}' for j in range(4)))

    def get_accuracy_score(self, outputs, labels):
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        return correct

    def accuracy(self, data_loader):
        correct = 0
        total = 0
        # since we're not training, we don't need to calculate the gradients for our outputs
        with torch.no_grad():
            for data in data_loader:
                # images, labels = data
                images, labels = data[0].to(get_device()), data[1].to(get_device())
                # calculate outputs by running images through the network
                outputs = self(images)
                # the class with the highest energy is what we choose as prediction
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        accuracy_score = {100 * correct // total}
        return accuracy_score

    def accuracy_in_each_class(self):
    # prepare to count predictions for each class
        correct_pred = {classname: 0 for classname in self.classes}
        total_pred = {classname: 0 for classname in self.classes}

        # again no gradients needed
        with torch.no_grad():
            for data in self.testloader:
                # images, labels = data
                images, labels = data[0].to(get_device()), data[1].to(get_device())
                outputs = self(images)
                _, predictions = torch.max(outputs, 1)
                # collect the correct predictions for each class
                for label, prediction in zip(labels, predictions):
                    if label == prediction:
                        correct_pred[self.classes[label]] += 1
                    total_pred[self.classes[label]] += 1

        # print accuracy for each class
        for classname, correct_count in correct_pred.items():
            accuracy = 100 * float(correct_count) / total_pred[classname]
            print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')



In [64]:

criterion = nn.CrossEntropyLoss()

pujan_net = PujanNet(data_set_name = "cifar_10", batch_size = 4, download_path = "./data", 
                     train_data_size = 40000, validation_data_size = 10000)
pujan_net = pujan_net.to(get_device())

train_results = pujan_net.train(epoch_number = 2, learning_rate = 0.01, momentum = 0.9, criterion = criterion, optimizer_name = "sgd")

print(train_results)
pujan_net.accuracy(pujan_net.testloader)

Files already downloaded and verified
Length of Dataset  50000
Length of Train set  40000
Length of Val set  10000
Files already downloaded and verified
Length of Testset  10000
[1,  2000] loss: 2.094
[1,  4000] loss: 1.973
[1,  6000] loss: 1.953
[1,  8000] loss: 1.946
[1, 10000] loss: 1.935
Train losses  [2.290104389190674, 2.283853530883789, 2.316251754760742, 2.2715108394622803, 2.357302188873291, 2.2413361072540283, 2.3786230087280273, 2.3488733768463135, 2.3483076095581055, 2.348104476928711, 2.2845847606658936, 2.3305938243865967, 2.3376524448394775, 2.2531704902648926, 2.3706212043762207, 2.287968397140503, 2.291968822479248, 2.2889795303344727, 2.292910099029541, 2.27066707611084, 2.2408499717712402, 2.290756940841675, 2.2833805084228516, 2.2580301761627197, 2.3212664127349854, 2.3062808513641357, 2.290480613708496, 2.3551182746887207, 2.354353666305542, 2.306015729904175, 2.3498809337615967, 2.3592007160186768, 2.363257884979248, 2.305318593978882, 2.2655234336853027, 2.319807

In [ ]:
# Experiments
# selecting the model
# CNN Architectures (How to choose CNN architecture ????)
# Number of Layers ??
# Batch Normalization
# Dropout

# Input data enhancements
# Data augmentation

# Hyper parameters
# Loss
# Optimizer
# Epoch
# Learning rate
# Batch size

